# Курс "Введение в анализ данных", весна 2018
## Семинар 3. Scikit-learn

На этом семинаре мы: 
- познакомимся с популярной библиотекой для машинного обучения [scikit-learn](http://scikit-learn.org/stable/);
- поговорим про оценку качества алгоритмов и кросс-валидацию;
- решим несколько простых задач машинного обучения с помощью scikit-learn.

scikit-learn — это библиотека...:
- с большинством популярных и не очень алгоритмов машинного обучения с __единым интерфейсом работы__;
- с большинством необходимым для машинного обучения вспомогательных утилит (преобразование признаков, метрики для оценки качества, ...);
- с подробной документацией и множеством примеров решения задач.

Можно сказать, что почти любой алгоритм, с которым вы столкнётесь (если это не связано со свежей статьёй или нейросетями), уже реализован в scikit-learn. Не бойтесь лезть в поисковик или [документацию](http://scikit-learn.org/stable/).

__Начнём.__

Импортируем необходимые библиотеки:

In [ ]:
%pylab inline
import pandas as pd

Посмотрим на разделы:

In [ ]:
from sklearn import <press tab>

- datasets — "игрушечные датасеты", функции для генерации или загрузки выборок
- feature_extraction — функции для извлечения признаков
- linear_model — линейный модели машинного обучения
- metrics — метрики для оценки качества алгоритмов
- preprocessing — преобразование признаков
- ...

Можно даже подходить к решению задач с помощью scikit-learn по следующей схеме. Хотя на самом деле она вам вряд ли понадобится (базовое представление будет и так в вашей голове).
![](sklearn_scheme.jpg)

Давайте решим несколько задач машинного обучения с помощью scikit-learn и убедимся, что разные виды задач решаются сходим образом. 

### Задача классификации

Воспользуемся стандартным набором данных для классификации цветков ириса на 3 класса.

In [ ]:
from sklearn.datasets import load_iris

iris = load_iris()
iris.keys()

In [ ]:
X = iris['data']
iris['data'].shape

То есть выборка состоит из 150 объектов и 4 признаков. Признаки:

In [ ]:
iris['feature_names']

Целевая переменная и названия классов:

In [ ]:
y = iris['target']
iris['target']

In [ ]:
iris['target_names']

Посмотрим на распределение двух признаков:

In [ ]:
x_index = 0
y_index = 1

# this formatter will label the colorbar with the correct target names
formatter = plt.FuncFormatter(lambda i, *args: iris.target_names[int(i)])

plt.scatter(iris.data[:, x_index], iris.data[:, y_index],
            c=iris.target, cmap=plt.cm.get_cmap('RdYlBu', 3))
plt.colorbar(ticks=[0, 1, 2], format=formatter)
plt.clim(-0.5, 2.5)
plt.xlabel(iris.feature_names[x_index])
plt.ylabel(iris.feature_names[y_index])
plt.show()

Заметим, что уже по этим двум признакам визуально можно резделить часть объектов на классы. 

Построим решающее дерево, классифицирующее объекты (не будем вдаваться в подробности этого метода). Импортируем его:

In [ ]:
from sklearn.tree import DecisionTreeClassifier

Обучим и сделаем предсказания:

In [ ]:
classifier = DecisionTreeClassifier()
classifier.fit(X, y)
y_predict = classifier.predict(X)

Посчитаем долю правильно классифицированных объектов:

In [ ]:
np.mean(y_predict == y)

Всё ли мы сделали верно?

Мы обучались и тестировались на одной и той же выборке. Наш алгоритм хорошо настроился на этой выборке и показал максимально возможный результат. Мы __переобучились__ на нашей обучающей выборке.

Однако в реальности нам вряд ли нужен алгоритм, который будет корректно классифицировать те же самые объекты, метки которых мы и так знаем. Поэтому в машинном обучении важно настраивать алгоритмы и оценивать их качество __на разных выборках__.

Разделим нашу выборку на 2 равных части: обучающую и контрольную выборки. Для этого воспользуемся функцией из scikit-learn.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=123)

Снова обучим алгоритм и сделаем предсказание уже на тестовой выборке:

In [ ]:
classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train)
y_predict = classifier.predict(X_test)

Оценим качество:

In [ ]:
np.mean(y_predict == y_test)

Теперь уже не все объекты верно классифицированы. В реальном мире почти нет задач, где можно получить идеальные предсказания с помощью машинного обучения.

Заметим, что в последнем эксперименте мы проигнорировали половину выборки для обучения и оценивали качество так же только по половине выборки. Хочется использовать всю выборку для оценки качества. Для этого вместо использования __отложенной выборки__ используется такая техника, как __кросс-валидация__.

![](cv.png)

Суть заключается в том, что мы делим выборку на несколько частей, для каждой из частей мы обучаем на оставшихся (за исключением текущей) частях алгоритм и оцениваем качество на выбранной части. Посчитанные метрики для каждой из частей усредняем.

Для кросс-валидации можно использовать функцию из scikit-learn. Разбиение можно задавать различными способами, но мы будем пользоваться самым простым: случайно на 2 части.

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
cross_val_score(classifier, X, y, scoring='accuracy', cv=2).mean()

То есть наш pipeline был следующим:

1. Загрузить данные;
2. Обработать их при надобности, визуализировать для понимания;
3. Разбить выборку на обучение и контоль (либо воспользовать кросс-валидацией);
4. Обучить некоторый алгоритм машинного обучения (будут изучаться на лекциях);
5. Оценить качество по некоторой метрике (будут изучаться на лекциях).

### Задача регрессии

Загрузим датасет, в котором необходимо предсказывать цену дома.

In [ ]:
from sklearn.datasets import load_boston
boston = load_boston()

In [ ]:
X = boston['data']
boston['data'].shape

То есть выборка состоит из 506 объектов и 13 признаков.

In [ ]:
boston['feature_names']

In [ ]:
X[0]

Целевая переменная и названия классов:

In [ ]:
y = boston['target']
y[:15]

Воспользуемся линейной регрессией (то есть выразим цену дома как линейную функцию вида $y_i = a_1 x_1 + a_2 x_2 + \dots + a_{13} x_{13}$, где $x_i$ — признаки дома). Оценивать качество будем с помощью среднеквадратичной ошибки (о функционалах ошибки будет позднее в лекциях). Выборку будем разбивать на 3 части.

In [ ]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression(normalize=True)

In [ ]:
cross_val_score(regressor, X, y, scoring='neg_mean_squared_error', cv=3).mean()

Попробуем воспользоваться другим алгоритмом — случайным лесом (несколько решающих деревьев, ответы которых усредняются).

In [ ]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor()
cross_val_score(regressor, X, y, scoring='neg_mean_squared_error', cv=3).mean()

Качество заметно улучшилось! 

У большинства алгоритмов в scikit-learn есть гиперпараметры. Их настройка может помочь алгоритмы ещё лучше настроиться на конкретную задачу и показать более высокое качество. 

Посмотрим на параметры случайного леса:

In [ ]:
RandomForestRegressor(<press shift+tab>)

Заметим, что решая задачу регрессии, мы делали то же самое, что и в классификации за исключением используемых методов и метрик для оценки качества. Scikit-learn позволяет работать с большинством алгоритмов с помощью одних и тех методов (fit, predict, ...). Это позволяет быстро экспериментировать с различными методами при решении конкретных задач.

### Классификация текстов

In [1]:
from sklearn.datasets import fetch_20newsgroups
data = fetch_20newsgroups()
data.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR', 'description'])

Посмотрим на пример объекта:

In [4]:
data['data'][1]

"From: guykuo@carson.u.washington.edu (Guy Kuo)\nSubject: SI Clock Poll - Final Call\nSummary: Final call for SI clock reports\nKeywords: SI,acceleration,clock,upgrade\nArticle-I.D.: shelley.1qvfo9INNc3s\nOrganization: University of Washington\nLines: 11\nNNTP-Posting-Host: carson.u.washington.edu\n\nA fair number of brave souls who upgraded their SI clock oscillator have\nshared their experiences for this poll. Please send a brief message detailing\nyour experiences with the procedure. Top speed attained, CPU rated speed,\nadd on cards and adapters, heat sinks, hour of usage per day, floppy disk\nfunctionality with 800 and 1.4 m floppies are especially requested.\n\nI will be summarizing in the next two days, so please add to the network\nknowledge base if you have done the clock upgrade and haven't answered this\npoll. Thanks.\n\nGuy Kuo <guykuo@u.washington.edu>\n"

Классы:

In [3]:
data['target_names']

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

Объект представляет собой текст с некоторыми элементами форматирования. Большинство алгоритмов машинного обучения не умеют работать напрямую с текстами. Необходимо представить текст в виде некоторого вектора, чтобы с полученной матрицей объекты-признаки воспользоваться некоторым алгоритмом классификации. Самый простой способ сделать это для текста — посчитать, сколько раз встречается каждое слово в каждом тексте.

Предварительно разделим выборку на обучающую и контрольную выборки:

In [ ]:
train, test, y_train, y_test = train_test_split(data['data'], data['target'], test_size=0.3)

В scikit-learn уже реализован класс для подсчёта встречаемости каждого слова в тексте. У него есть различные параметры, связанные с предобработкой текста.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(stop_words='english')
train_vectors = vectorizer.fit_transform(train)
test_vectors = vectorizer.transform(test)

Так как слов в языке довольно много, при этом малое количество из них встречается в каждом конкретном тексте, то в полученной матрице, где $i,j$ позиции находится число "сколько раз в тексте $i$ встречается слово $j$, возникает много нулевых элементов. Такие матрицы называют разреженными и их удобнее хранить в памяти в ином формате.

In [ ]:
train_vectors

In [ ]:
train_vectors[0].todense()

В процессе кодирования текста мы получили отображение слов в индексы:

In [ ]:
vectorizer.vocabulary_

Воспользуемся логистической регрессией (линейный алгоритм для классификации), обучим его и оценим качество (долю верно предсказанных категорий) на отложенной выборке.

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
classifier = LogisticRegression()
classifier.fit(train_vectors, y_train)
y_predict = classifier.predict(test_vectors)

In [ ]:
np.mean(y_predict == y_test)

Опять же мы делали все те же самые действия и использовали почти только один scikit-learn.

Нам так же могло понадобиться:
- закодировать строковый категориальный признак числами (sklearn.preprocessing.LabelEncoding)
- воспользовать одной из реализованных метрик для оценки качества (sklearn.metrics)
- ...